# Taxi Demand Forecasting

**Project Description**

The company "Cool Taxi" has collected historical data on taxi orders at airports. To attract more drivers during peak hours, it is necessary to forecast the number of taxi orders for the next hour. Build a model for such prediction.

The RMSE metric value on the test sample should not exceed 48.

**Data Description**

The data is located in the file `/datasets/taxi.csv`.

The number of orders is in the column `num_orders`.

**Work Plan**
1. Load the data and resample it by one hour.
2. Analyze the data.
3. Train different models with various hyperparameters. Make a test sample of size 10% of the original data.
4. Check the data on the test sample and draw conclusions.

In [1]:
import pandas as pd
import numpy as np
import optuna
import plotly.express as px

from collections import defaultdict
from IPython.display import display

from ydata_profiling import ProfileReport
from fast_ml import eda
from statsmodels.tsa.seasonal import seasonal_decompose

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

from sklearn.model_selection import TimeSeriesSplit

from sklearn.metrics import mean_squared_error

In [2]:
FIG_WIDTH = 10 * 100
FIG_HEIGHT = 5 * 100
RANDOM_SEED = 42

In [3]:
try:
    raw_taxi_orders = pd.read_csv('taxi.csv')
except:
    raw_taxi_orders = pd.read_csv('/datasets/taxi.csv')

## Exploratory Data Analysis

Let's examine the main dependencies in the data before we use them in machine learning algorithms.

Summary Table:

In [4]:
display(eda.df_info(raw_taxi_orders))

,data_type,data_type_grp,num_unique_values,sample_unique_values,num_missing,perc_missing
datetime,object,Categorical,26496,"[2018-03-01 00:00:00, 2018-03-01 00:10:00, 201...",0,0.0
num_orders,int64,Numerical,81,"[9, 14, 28, 20, 32, 21, 7, 5, 17, 12]",0,0.0


Numerical distributions:

In [5]:
display(round(raw_taxi_orders.describe().T, 2))

,count,mean,std,min,25%,50%,75%,max
num_orders,26496.0,14.07,9.21,0.0,8.0,13.0,19.0,119.0


And a detailed report:

In [6]:
ProfileReport(raw_taxi_orders, tsmode=True).to_widgets()

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Our dataset is quite straightforward: for each 10-minute interval, we have the number of taxi orders. There are no missing values in the dataset, but there is a small (2%) number of zero values. We'll drop them to simplify the task for machine learning.

# Data Preparation for ML

Since the data is already quite clean, the preprocessing will be relatively quick:

1. We'll create new columns with features.
2. We'll trim zero values.

In [7]:
df_taxi_orders = (
    raw_taxi_orders
    .loc[lambda df: df.num_orders > 0]
    .reset_index(drop=True)
    .assign(datetime=lambda df: pd.to_datetime(df.datetime))
    .set_index('datetime')
    .resample('1H').sum()
    .assign(
        hour=lambda df: df.index.hour,
        day_of_week=lambda df: df.index.dayofweek,
        is_weekend=lambda df: df.day_of_week.isin([5, 6]).astype(int),
        day_of_month=lambda df: df.index.day,
        month=lambda df: df.index.month,
        year=lambda df: df.index.year,
        lag_1=lambda df: df.num_orders.shift(1),
        mean_rol_3=lambda df: df.lag_1.rolling(window=3).mean(),
        std_rol_3=lambda df: df.lag_1.rolling(window=3).std()
    )
    .dropna()
)

print('Index is monotonically increasing:', df_taxi_orders.index.is_monotonic_increasing)
display(df_taxi_orders.head())

Index is monotonically increasing: True


,num_orders,hour,day_of_week,is_weekend,day_of_month,month,year,lag_1,mean_rol_3,std_rol_3
datetime,,,,,,,,,,
2018-03-01 03:00:00,66,3,3,0,1,3,2018,71.0,93.333333,27.465129
2018-03-01 04:00:00,43,4,3,0,1,3,2018,66.0,74.000000,9.848858
2018-03-01 05:00:00,6,5,3,0,1,3,2018,43.0,60.000000,14.933185
2018-03-01 06:00:00,12,6,3,0,1,3,2018,6.0,38.333333,30.270998
2018-03-01 07:00:00,15,7,3,0,1,3,2018,12.0,20.333333,19.857828


Now we can take a look at more detailed plots; perhaps we'll notice some dependencies in the data.

In [8]:
ProfileReport(df_taxi_orders, tsmode=True).to_widgets()

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

And break them down into components.

In [9]:
def generate_plot(y_column, title, xlim=None):
    """
    Generate a line plot for the given column with specified title and x-axis limits.

    Parameters:
    - y_column (str): The column name in df_temp to be plotted on the y-axis.
    - title (str): The title of the plot.
    - xlim (list, optional): A list of two datetime values specifying the x-axis range. Defaults to None.

    Returns:
    - None: Displays the plot.
    """
    fig = px.line(
        df_temp.melt(id_vars='datetime', value_vars=[y_column], var_name='component', value_name='value'),
        x='datetime',
        y='value',
        color='component',
        title=title,
        template='plotly_white',
        width=FIG_WIDTH, height=1.5*FIG_HEIGHT
    )
    if xlim:
        fig.update_xaxes(range=xlim)
    fig.update_layout(legend=dict(orientation='h'))
    fig.show()

In [10]:
temp = seasonal_decompose(df_taxi_orders.num_orders, model='additive', period=24)

df_temp = pd.DataFrame({
    'datetime': df_taxi_orders.index,
    'trend': temp.trend,
    'seasonal': temp.seasonal,
    'residual': temp.resid
})

generate_plot('trend', 'Trend over time')
generate_plot(
    'seasonal', 'Seasonal variation over time',
    [df_temp.datetime.iloc[0], df_temp.datetime.iloc[0] + pd.Timedelta(days=3)]
)
generate_plot('residual', 'Residuals over time')

Here it gets a bit more interesting:

1. We clearly have a fluctuating trend component that is increasing. This is good - it likely means our business is growing because we're seeing more orders.

2. There is also a seasonal component, although it's worth noting that its period is measured in hours compared to the days of the trend. The seasonality looks reasonable, considering the taxi operates at an airport: during nighttime, we observe more orders than in the morning and afternoon.

3. Finally, the remainder component appears random, but it's worth noting how it sharply increases after August. Most likely, our models will also suffer a loss in quality during this period.

## ML Models

Let's create and train several models. First, let's split the data into `train` and `test` sets. We'll create a function for this to directly assign the data to the dataframe.

In [11]:
def split_data(df: pd.DataFrame, target_column: str, test_size: float, shuffle=False):
    """
    Split a DataFrame into training and testing datasets.

    This function accepts a DataFrame, the name of the target column, and the proportion of the data 
    to be included in the test split. It returns four DataFrames: the training features, the training target, 
    the testing features, and the testing target. The target datasets are DataFrames with a single column rather 
    than Series objects.

    Args:
    - df (pd.DataFrame):
        The DataFrame to split. This DataFrame should include both the features and the target.

    - target_column (str): 
        The name of the target column. This column will be separated from the features and returned 
        in the target DataFrames.

    - test_size (float):
        The proportion of the data to include in the test split. For example, if `test_size` is 0.3, 
        30% of the data will be used for the test split, and the rest will be used for the training split.
        
    - shuffle (boolean):
        A flag to shuffle (True) or not (False) the data when splitting into train and test.

    Returns
    - list of pd.DataFrame:
        A list containing four DataFrames: the training features, the training target, 
        the testing features, and the testing target.
    """
    df_train, df_test = train_test_split(
        df, test_size=test_size, random_state=RANDOM_SEED, shuffle=shuffle
    )
    
    ftr_train = df_train.drop(target_column, axis=1)
    tgt_train = df_train[[target_column]]
    ftr_test = df_test.drop(target_column, axis=1)
    tgt_test = df_test[[target_column]]
    
    return [ftr_train, tgt_train, ftr_test, tgt_test]

Let's save the datasets.

In [12]:
dct_splits = split_data(df_taxi_orders, 'num_orders', 0.1)

dct_splits = {
    'train': {'features': dct_splits[0], 'target': dct_splits[1]},
    'test': {'features': dct_splits[2], 'target': dct_splits[3]}
}

print(
    'Test to full sample size:', 
    round(100 * dct_splits['test']['target'].shape[0] / df_taxi_orders.shape[0], 2),
    '%'
)

Test to full sample size: 10.02 %


The datasets are split as expected.

Let's set up a `study` for `optuna` - it will find optimal models for us.

In [13]:
def optimize_regressors(ftr_train, tgt_train, n_trials: int):
    """
    Trains and optimizes regression models using Optuna.

    Args:
    - ftr_train, tgt_train: Training features and target.
    - n_trials (int): The number of trials for Optuna optimization.

    Returns:
    - An Optuna study object containing the optimal model and its parameters.
    """
    # Ensure target is 1-d vector
    tgt_train = tgt_train.values.ravel()
    
    # Define TimeSeriesSplit object
    tscv = TimeSeriesSplit(n_splits=3)

    def get_regressor(trial):
        regressors = {
            'LinearRegression': LinearRegression(),
            'RandomForest': RandomForestRegressor(
                max_depth=trial.suggest_int('max_depth', 1, 100),
                n_estimators=trial.suggest_int('n_estimators', 100, 1000),
                random_state=RANDOM_SEED
            ),
            'CatBoost': CatBoostRegressor(
                iterations=trial.suggest_int('iterations', 100, 1000),
                learning_rate=trial.suggest_float('learning_rate', 0.01, 0.3),
                logging_level='Silent',
                random_state=RANDOM_SEED
            ),
            'LGBM': LGBMRegressor(
                max_depth=trial.suggest_int('max_depth', 1, 50),
                n_estimators=trial.suggest_int('n_estimators', 100, 1000),
                learning_rate=trial.suggest_float('learning_rate', 0.01, 0.3),
                random_state=RANDOM_SEED
            )
        }
        regressor_name = trial.suggest_categorical('regressor', list(regressors.keys()))
        return regressors[regressor_name]

    def objective(trial):
        """
        Objective function for Optuna optimization. Computes the RMSE for a given regressor.

        Args:
        - trial (optuna.Trial): 
            A trial is a process of evaluating an objective function. 
            This object is passed to an objective function and provides interfaces to 
            suggest hyperparameters.

        Returns:
        - float:
            Root Mean Squared Error (RMSE) of the regressor's predictions.
        """
        regressor_obj = get_regressor(trial)
        
        # Manually loop through the splits and collect predictions
        predictions = []
        targets = []
        for idx_train, idx_test in tscv.split(ftr_train):
            X_train, X_test = ftr_train.iloc[idx_train], ftr_train.iloc[idx_test]
            y_train, y_test = tgt_train[idx_train], tgt_train[idx_test]
            
            regressor_obj.fit(X_train, y_train)
            preds = regressor_obj.predict(X_test)
            
            predictions.extend(preds)
            targets.extend(y_test)
        
        return np.sqrt(mean_squared_error(targets, predictions))

    optuna.logging.set_verbosity(optuna.logging.WARNING)
    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=RANDOM_SEED))
    study.optimize(objective, n_trials=n_trials)
    
    return study

Let's take the results of the `study`.

In [14]:
study = optimize_regressors(
    dct_splits['train']['features'],
    dct_splits['train']['target'],
    n_trials=20
)

Let's see how the models compare to each other.

In [15]:
best_params = study.best_params
formatted_params = "\n".join([f"  {key}: {value}" for key, value in best_params.items()])
print(f"Best params:\n{formatted_params}")

fig = optuna.visualization.plot_optimization_history(study)
fig.update_layout(
    legend=dict(orientation='h'),
    template='plotly_white',
    width=FIG_WIDTH, height=FIG_HEIGHT
)
fig.show()

fig = optuna.visualization.plot_slice(study)
fig.update_layout(
    legend=dict(orientation='h'),
    template='plotly_white',
    width=FIG_WIDTH, height=FIG_HEIGHT
)
fig.update_xaxes(tickangle=-90)
fig.show()

Best params:
  max_depth: 74
  n_estimators: 339
  iterations: 345
  learning_rate: 0.10709060110473853
  regressor: CatBoost


The optimization process shows that, given the available data and the considered range of hyperparameters, the `CatBoost` regressor provides the lowest RMSE error on the training data. We expect that this model will show the best results on unseen data among all the models and hyperparameters considered.

## Results Check

As a final step, let's check how the model performs on the test data, as well as examine the predictions we obtained.

First, let's train the best model.

In [16]:
model = CatBoostRegressor(
    iterations=study.best_params['iterations'],
    learning_rate=study.best_params['learning_rate'],
    logging_level='Silent',
    random_state=RANDOM_SEED
)

model.fit(dct_splits['train']['features'], dct_splits['train']['target']);

Let's calculate the predictions and examine the metrics.

In [17]:
for sample in ['train', 'test']:
    # Convert predictions array to DataFrame
    dct_splits[sample]['prediction'] = pd.DataFrame(
        model.predict(dct_splits[sample]['features']),
        columns=['num_orders'],
        index=dct_splits[sample]['target'].index
    )
    
    rmse = np.sqrt(
        mean_squared_error(
            dct_splits[sample]['target'], dct_splits[sample]['prediction']
        )
    )
    
    print(f"RMSE on {sample} sample: {rmse:.3f}")

RMSE on train sample: 15.506
RMSE on test sample: 45.789


Out of curiosity, let's visualize the predictions on graphs.

In [18]:
df_temp = (
    pd.concat([
        dct_splits[split][data_type]
        .assign(
            split=split, is_predicted=(data_type == 'prediction')
        )
        for split in ['train', 'test']
        for data_type in ['target', 'prediction']
    ])
    .pipe(
        lambda df: df.assign(rolling_3h=df.num_orders.rolling(3).mean())
    )
)

display(df_temp.head())

,num_orders,split,is_predicted,rolling_3h
datetime,,,,
2018-03-01 03:00:00,66.0,train,False,NaN
2018-03-01 04:00:00,43.0,train,False,NaN
2018-03-01 05:00:00,6.0,train,False,38.333333
2018-03-01 06:00:00,12.0,train,False,20.333333
2018-03-01 07:00:00,15.0,train,False,11.000000


In [19]:
fig_config = {
    'train': {
        'x_range': [df_temp.index.min(), df_temp.index.min() + pd.Timedelta(days=14)],
        'y_range': [0, 150]
    },
    'test': {}
}

for split, config in fig_config.items():
    fig = px.line(
        df_temp[df_temp.split == split].reset_index(),
        x='datetime',
        y='rolling_3h',
        color='is_predicted',
        title=f'True and predicted values for {split} sample',
        template='plotly_white',
        width=FIG_WIDTH, height=FIG_HEIGHT
    )
    fig.update_xaxes(range=config.get('x_range'))
    fig.update_yaxes(range=config.get('y_range'))
    fig.update_layout(legend=dict(orientation='h'))
    fig.show()


## Conclusions

Based on the analysis and modeling results, the following conclusions can be made:

1. Taxi order data exhibits clear seasonality and a growing trend, indicating successful business development. Particularly active orders are observed during nighttime hours.
  
2. The CatBoost model with specific hyperparameters showed the best results among those considered, as evidenced by the RMSE value on the training data.

3. Despite good performance on the training set, the model demonstrates a higher error on the test set, which may indicate overfitting or changes in the data characteristics.

4. The residual component of the model shows an increase in error after August, which may require further analysis or model correction for this time period.

5. Overall, the model can be used for taxi order forecasting, but careful consideration should be given to the forecasting horizon, and the model should be regularly validated with up-to-date data.